In [42]:
# !pip install resemblyzer librosa scikit-learn soundfile --quiet
!pip install speechbrain --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 46.0 MB/s eta 0:00:00


In [17]:
from google.colab import files, os

os.makedirs("voices_opus_me", exist_ok=True)
os.makedirs("voices_opus_others", exist_ok=True)
print("Upload your .opus voice files:")
uploaded = files.upload()

for name in uploaded.keys():
    os.rename(name, f"voices_opus_me/{name}")

print("upload other .opus files")
uploaded = files.upload()

for name in uploaded.keys():
    os.rename(name, f"voices_opus_others/{name}")

Upload your .opus voice files:


Saving my_1.opus to my_1.opus
Saving my_2.opus to my_2.opus
Saving my_3.opus to my_3.opus
Saving my_4.opus to my_4.opus
upload other .opus files


Saving A_voice.opus to A_voice.opus
Saving anas.opus to anas.opus
Saving baba.opus to baba.opus
Saving faiez.opus to faiez.opus


In [ ]:
from google.colab import files
import os


for src, dst in [("voices_opus_me", "voices_wav_me"), ("voices_opus_others", "voices_wav_others")]:
    os.makedirs(dst, exist_ok=True)
    for file in os.listdir(src):
        if file.endswith(".opus"):
            wav_name = file.replace(".opus", ".wav")
            os.system(f"ffmpeg -i {src}/{file} -ar 16000 -ac 1 {dst}/{wav_name}")


print("All my .opus files converted to 16kHz mono .wav format in voices_wav_me/")

All my .opus files converted to 16kHz mono .wav format in voices_wav_me/


In [43]:
from speechbrain.pretrained import EncoderClassifier
import torch
import numpy as np
import torchaudio
import os
from glob import glob

# Load pretrained ECAPA-TDNN model
encoder = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

X, y = [], []

def embed_folder(folder, label):
    for f in glob(f"{folder}/*.wav"):
        # Load and preprocess audio
        signal, fs = torchaudio.load(f)

        # Convert to mono if stereo
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)

        # Resample to 16kHz if needed
        if fs != 16000:
            resampler = torchaudio.transforms.Resample(fs, 16000)
            signal = resampler(signal)

        # Extract ECAPA embedding
        emb = encoder.encode_batch(signal).squeeze().detach().cpu().numpy()

        X.append(emb)
        y.append(label)

    print(f"Processed {len(glob(f'{folder}/*.wav'))} files from {folder}")

# Run for both folders
embed_folder("voices_wav_me", "me")
embed_folder("voices_wav_others", "not_me")

X, y = np.array(X), np.array(y)
print("ECAPA embeddings ready:", X.shape, len(y))


/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoi

hyperparams.yaml: 0.00B [00:00, ?B/s]

INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered parameter transfer hook for _load
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/autocast.py:188: Fut

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


...

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/label_encoder.txt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /root/.cache/huggingface/hub/models--speechb

Processed 4 files from voices_wav_me
Processed 4 files from voices_wav_others
ECAPA embeddings ready: (8, 192) 8


In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
joblib.dump(clf, "voice_auth.pkl")
print("Model saved as voice_auth.pkl")

Accuracy: 1.0
Model saved as voice_auth.pkl


In [45]:
from google.colab import files
files.download("voice_auth.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Voice verification testing via loaded/trained model for COLAB

In [58]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving my_test.opus to my_test.opus


In [ ]:
# ==============================================
# Voice Authentication Test Script (ECAPA)
# ==============================================
from speechbrain.pretrained import EncoderClassifier
import torch, torchaudio, numpy as np, joblib
from google.colab import files

# ------------------------------------------------
# Load model and classifier
# ------------------------------------------------
print(" Loading classifier and ECAPA encoder...")
clf = joblib.load("voice_auth.pkl")
encoder = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
print("Model loaded successfully.\n")

# ------------------------------------------------
# Upload test voice
# ------------------------------------------------
# print("Upload a test voice file (.wav):")
# uploaded = files.upload()
# filename = list(uploaded.keys())[0]
print(f"Received: {filename}")

# ------------------------------------------------
# Preprocess + Embed the uploaded audio
# ------------------------------------------------
signal, fs = torchaudio.load(filename)

# Convert to mono if stereo
if signal.shape[0] > 1:
    signal = torch.mean(signal, dim=0, keepdim=True)

# Resample to 16 kHz if needed
if fs != 16000:
    resampler = torchaudio.transforms.Resample(fs, 16000)
    signal = resampler(signal)

# Generate ECAPA embedding
embedding = encoder.encode_batch(signal).squeeze().detach().cpu().numpy()

# ------------------------------------------------
# Predict using your trained classifier
# ------------------------------------------------
pred = clf.predict([embedding])[0]
proba = clf.predict_proba([embedding])[0]

print("\n Prediction Results:")
print(f"Classified as: {pred}")
print(f"Confidence: {proba.max():.2f}")

# ------------------------------------------------
# Decision logic
# ------------------------------------------------
if pred == "me" and proba.max() > 0.75:
    print("\nVoice verified! Access granted.")
else:
    print("\nAccess denied. Voice does not match.")


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


📂 Loading classifier and ECAPA encoder...


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Fetching files for pretraining (no collection directory set)
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/mean_var_norm_emb.ckpt
INFO:sp

✅ Model loaded successfully.

📁 Received: my_test.opus

🔍 Prediction Results:
🧠 Classified as: me
📊 Confidence: 1.00

✅ Voice verified! Access granted.
